In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

In [ ]:
def get_english_dictionary():
    dict_url = "https://github.com/dwyl/english-words/raw/master/words_alpha.txt"
    response = requests.get(dict_url)
    # splitlines() automatically handles \n, \r\n, and \r
    words = set(word.strip().lower() for word in response.text.splitlines() if word.strip())
    return words

def return_words(web_page):
    """Count English words on a web page."""
    try:
        # Fetch and parse the web page
        response = requests.get(web_page)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract text and clean it
        text = soup.get_text()
        text = text.lower()
        text = re.sub(r'[",:.\\n]', '', text)
        
        # Split into words and count those in the dictionary
        page_words = text.split()

        english_count = sum(1 for word in page_words if word in english_dictionary)
        return english_count
    except:
        return 0

def main():
    """Main function to process URLs and update Google Sheet."""
    # Set up Google Sheets authentication
    # You'll need to create a service account and download credentials.json
    scopes = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]
    
    credentials = Credentials.from_service_account_file(
        'credentials.json',
        scopes=scopes
    )
    
    gc = gspread.authorize(credentials)
    
    # Open the sheet
    sheet_url = 'YOUR_GOOGLE_SHEET_URL_HERE'
    spreadsheet = gc.open_by_url(sheet_url)
    
    # Read data from the sheet
    worksheet = spreadsheet.sheet1  # Assuming URLs are in the first sheet
    page_data = pd.DataFrame(worksheet.get_all_records())
    
    # Get English dictionary
    global english_dictionary
    english_dictionary = get_english_dictionary()
        
    # Process each URL and count words
    page_data['words'] = page_data['page_location'].apply(return_words)
    
    print(page_data)
    
    # Write results to a new sheet
    try:
        output_sheet = spreadsheet.worksheet('output')
    except:
        output_sheet = spreadsheet.add_worksheet(title='output', rows=100, cols=20)
    
    set_with_dataframe(output_sheet, page_data)
    print("Processing complete. Results written to 'output' sheet.")

if __name__ == "__main__":
    main()